In [1]:
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from agent import *
from selection import *
from utils import *
from env import Environment
from simulation import constructAgent
from main import Config

In [2]:
torch.cuda.is_available()

True

In [3]:
MULTI_SELECTION_METHOD = 'A2C'
config = {
    'reward': 3,
    'sucker': 0,
    'temptation': 5,
    'punishment': 1,
    'n_episodes': 50000,
    'discount': 0.99,
    'play_epsilon': 1,
    'select_epsilon': 1,
    'epsilon_decay': 0.999,
    'min_epsilon': 0.01,
    'alpha': 0.1,
    'n_actions': 2,
    'h': 10,
    'state_repr': 'bi-repr',
    'batch_size': 64,
    'learning_rate': 1e-3,
}
config = Config(config)
env = Environment(config)
seed_everything()

In [4]:
agents = {}
index = 0
with HiddenPrints():
#     for _ in range(5):
#         agents[index] = constructAgent('TitForTat', config)
#         index += 1
#     for _ in range(5):
#         agents[index] = constructAgent('LSTM', config)
#         index += 1
    for _ in range(10):
        agents[index] = constructAgent('LSTMQN', config)
        index += 1

agents = a2c_selection(config, agents, env)

for n in range(len(agents)):
    print('Agent{}: name:{}  final score:{}  play time:{}  times to play D:{}  ratio: {}  faced D ratio: {}'
        .format(n, agents[n].name, agents[n].running_score,
        len(agents[n].own_memory[:agents[n].play_times]), list(agents[n].own_memory[:agents[n].play_times]).count(1),
                list(agents[n].own_memory[:agents[n].play_times]).count(1)/len(agents[n].own_memory[:agents[n].play_times]),
                list(agents[n].opponent_memory[:agents[n].play_times]).count(1)/len(agents[n].opponent_memory[:agents[n].play_times])))
print('The reward for total society: {}'.format(env.running_score/len(agents)))

100%|███████████████████████████████████| 50000/50000 [5:29:18<00:00,  2.53it/s]


Agent0: name:LSTMQN  final score:158.13956897710779  play time:10339  times to play D:7449  ratio: 0.7204758680723474  faced D ratio: 0.7861495309024084
Agent1: name:LSTMQN  final score:184.57084193321978  play time:9244  times to play D:7761  ratio: 0.8395716140199048  faced D ratio: 0.7324751189961056
Agent2: name:LSTMQN  final score:183.39681143051345  play time:7746  times to play D:6693  ratio: 0.8640588690937258  faced D ratio: 0.7616834495223341
Agent3: name:LSTMQN  final score:230.2679571738217  play time:7784  times to play D:6747  ratio: 0.8667780061664954  faced D ratio: 0.795349434737924
Agent4: name:LSTMQN  final score:248.95217059029278  play time:9852  times to play D:7948  ratio: 0.806739748274462  faced D ratio: 0.7544660982541616
Agent5: name:LSTMQN  final score:214.14454770963712  play time:13663  times to play D:9961  ratio: 0.7290492571177632  faced D ratio: 0.7959452536046256
Agent6: name:LSTMQN  final score:196.3550175451667  play time:9895  times to play D:7570 

In [6]:
def get_index_from_action(action, idx):
    scale = lambda x: x+1 if x>=idx else x
    return list(map(scale, action))

for idx in agents:
    transitions = agents[idx].SelectionMemory.memory
    _, actions, rewards, _ = zip(*transitions)
    actions = get_index_from_action(np.array(
        , dtype=int), idx)
    actions, rewards = np.array(actions), np.array(rewards)
    print(f'Agent {idx}:', end='')
    values, counts = np.unique(actions, return_counts=True)
    print(f' opponent_idx: {list(values)}, counts: {list(counts)} ', end='')
    dict_idx = {x: rewards[np.where(actions == x)] for x in values}
    print(f'rewards: {[np.mean(y) for _, y in dict_idx.items()]}')
    # print(f'rewards: {[np.std(y) for _, y in dict_idx.items()]}')
# plt.plot(actions)
# plt.show()
# print(rewards)

Agent 0: opponent_idx: [2, 3, 4, 5, 6, 7, 8, 9, 10], counts: [3, 10, 3, 3, 4842, 7, 2, 21, 9] rewards: [2.0, 1.4, 0.3333333333333333, 2.0, 1.9409334985543163, 3.857142857142857, 0.5, 2.0, 2.7777777777777777]
Agent 1: opponent_idx: [0, 3, 4, 5, 6, 7, 8, 9, 10], counts: [1088, 185, 122, 1008, 177, 630, 296, 1102, 522] rewards: [2.7895220588235294, 1.1891891891891893, 1.459016393442623, 1.9841269841269842, 1.3502824858757063, 2.2587301587301587, 1.5304054054054055, 2.515426497277677, 1.4923371647509578]
Agent 2: opponent_idx: [0, 1, 4, 5, 6, 7, 8, 9, 10], counts: [514, 371, 680, 304, 339, 747, 131, 1195, 621] rewards: [2.017509727626459, 1.4285714285714286, 1.7779411764705881, 1.388157894736842, 1.7817109144542773, 2.4939759036144578, 1.1526717557251909, 2.102092050209205, 1.7278582930756843]
Agent 3: opponent_idx: [0, 1, 2, 5, 6, 7, 8, 9, 10], counts: [508, 114, 367, 1607, 192, 1404, 249, 385, 195] rewards: [2.604330708661417, 1.2719298245614035, 1.2915531335149864, 1.7504667081518357, 1

In [7]:
agents = {}
index = 0
with HiddenPrints():
#     for _ in range(5):
#         agents[index] = constructAgent('TitForTat', config)
#         index += 1
    for _ in range(5):
        agents[index] = constructAgent('LSTM', config)
        index += 1
    for _ in range(5):
        agents[index] = constructAgent('LSTMQN', config)
        index += 1

agents = a2c_selection(config, agents, env)

for n in range(len(agents)):
    print('Agent{}: name:{}  final score:{}  play time:{}  times to play D:{}  ratio: {}  faced D ratio: {}'
        .format(n, agents[n].name, agents[n].running_score,
        len(agents[n].own_memory[:agents[n].play_times]), list(agents[n].own_memory[:agents[n].play_times]).count(1),
                list(agents[n].own_memory[:agents[n].play_times]).count(1)/len(agents[n].own_memory[:agents[n].play_times]),
                list(agents[n].opponent_memory[:agents[n].play_times]).count(1)/len(agents[n].opponent_memory[:agents[n].play_times])))
print('The reward for total society: {}'.format(env.running_score/len(agents)))

100%|███████████████████████████████████| 50000/50000 [5:28:21<00:00,  2.54it/s]


Agent0: name:LSTM  final score:294.92829254612724  play time:24348  times to play D:10960  ratio: 0.450139641859701  faced D ratio: 0.45313783472975194
Agent1: name:LSTM  final score:301.0213129579115  play time:10214  times to play D:1434  ratio: 0.14039553553945566  faced D ratio: 0.14538868220090073
Agent2: name:LSTM  final score:195.88937159971852  play time:6236  times to play D:4553  ratio: 0.7301154586273252  faced D ratio: 0.7429441949967928
Agent3: name:LSTM  final score:234.79067014047223  play time:14991  times to play D:8108  ratio: 0.5408578480421586  faced D ratio: 0.5346541258088187
Agent4: name:LSTM  final score:301.22932719184706  play time:8214  times to play D:1018  ratio: 0.12393474555636717  faced D ratio: 0.11699537375213051
Agent5: name:LSTMQN  final score:294.73145002281825  play time:5428  times to play D:2432  ratio: 0.44804716285924834  faced D ratio: 0.42962417096536476
Agent6: name:LSTMQN  final score:274.745324608607  play time:8178  times to play D:3552  

In [10]:
def get_index_from_action(action, idx):
    scale = lambda x: x+1 if x>=idx else x
    return list(map(scale, action))

for idx in agents:
    transitions = agents[idx].SelectionMemory.memory
    _, actions, rewards, _ = zip(*transitions)
    actions = np.array(actions, dtype=int)
    actions, rewards = np.array(actions), np.array(rewards)
    print(f'Agent {idx}:', end='')
    values, counts = np.unique(actions, return_counts=True)
    print(f' opponent_idx: {list(values)}, counts: {list(counts)} ', end='')
    dict_idx = {x: rewards[np.where(actions == x)] for x in values}
    print(f'rewards: {[np.mean(y) for _, y in dict_idx.items()]}')

Agent 0: opponent_idx: [1, 2, 3, 4, 5, 6, 7, 8, 9], counts: [10, 54, 2, 6, 1, 87, 4653, 88, 8] rewards: [1.7, 2.5, 0.5, 1.1666666666666667, 1.0, 1.8620689655172413, 1.6898774983881366, 2.3181818181818183, 2.375]
Agent 1: opponent_idx: [0, 2, 3, 4, 5, 6, 7, 8, 9], counts: [4940, 18, 9, 2, 5, 2, 14, 3, 8] rewards: [2.776923076923077, 2.5, 2.888888888888889, 2.0, 0.8, 4.0, 1.6428571428571428, 2.0, 1.25]
Agent 2: opponent_idx: [0, 1, 3, 4, 5, 6, 7, 8, 9], counts: [18, 10, 5017, 10, 3, 2, 3, 1, 10] rewards: [1.6666666666666667, 1.9, 1.2864261510863066, 1.1, 1.6666666666666667, 3.0, 3.0, 3.0, 1.6]
Agent 3: opponent_idx: [0, 1, 2, 4, 5, 6, 7, 8, 9], counts: [8, 26, 15, 3020, 11, 33, 59, 1680, 64] rewards: [2.0, 2.1538461538461537, 1.8, 2.863907284768212, 1.7272727272727273, 2.4242424242424243, 1.0847457627118644, 1.581547619047619, 2.84375]
Agent 4: opponent_idx: [0, 1, 2, 3, 5, 6, 7, 8, 9], counts: [10, 5068, 20, 11, 1, 9, 3, 4, 8] rewards: [3.1, 2.838989739542226, 3.0, 2.5454545454545454, 1